In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from datetime import datetime, timedelta
import pickle
import cartopy.crs as ccrs
import cartopy.feature as cfeature

plt.rcParams['legend.fontsize'] = 10
plt.rcParams.update({'figure.autolayout': True})

## Strain rate field by least-square collocation method

### Compute velocity from ELTM coefficients
The non-periodic component of east or north displacement, $u$, as a function of the number of days from 2011/04/11, is expressed in terms of the ELTM coefficients:

$ u(d) = pd + q + a\log\left( 1+\frac{2(d+21)}{365} \right) $, where $p$, $q$ and $a$ are 0-, 1- and 10-th coefficients, and 21 is the number of days since the March 11, 2011 Tohoku earthquake until Apritl 1, 2011.

Velocity can be derived as 
$ v(d) = p + \frac{ka}{1+k(d+21)}$, where $k = 2/365$ (i.e., the inverse of half a year).

### Loading Data
If a pickled data object exists, load and use it as an instance of `GPS_Stations` class named `Korea_GPS_Stations`.
The loaded pickle file should contain 55 stations.


In [ ]:
try: 
    with open( 'Korea_GPS_Stations_coeff_cov_20110401_20161231.p', 'rb' ) as handle:
        Korea_GPS_CoeffCov = pickle.load(handle)
        # This is a dictionary with 55 stations as keys
        # Each item is a dictionary with 4 keys, ecoeff, ecov, ncoeff, ncov.
        print(Korea_GPS_CoeffCov['DAEJ']['ecoeff'])
except FileNotFoundError:
    print("Failed to load the pickled data. Generate ELTM coefficients using GPSdata.ipynb.")

### Compute $\mathbf{M}$ matrix for each point
A strain rate tensor ($\boldsymbol{\varepsilon}$) at a location is the spatially linear rate of change of velocity at that location. In other words, velocity at a GPS station ($\mathbf{v}$) can be approximated as the sum of a translation vector, $\mathbf{t}$, and the product of strain rate and $\mathbf{X} = \mathbf{x}^{(i)} - \mathbf{x}$, where $\mathbf{x}^{(i)}$ is the position vector of the $i$-th station and $\mathbf{x}$ is the current location of interest:
$ \mathbf{t} + \boldsymbol{\varepsilon} \mathbf{X} = \mathbf{v}$.

Since we have multiple GPS stations and cannot expect the relationship to hold exactly for all of them, we try to find $\mathbf{t}$ and $\boldsymbol{\varepsilon}$ in the least square sense.
$ \mathbf{t}^{(i)} + \boldsymbol{\varepsilon} \mathbf{X}_{i}^{(i)} = \mathbf{v}^{(i)}$, where $\mathbf{X}^{(i)}$ is the position vector of the $i$-th GPS station minus the position vector of a grid point, and $\mathbf{v}^{(i)}$ is the GPS velocity at the $i$-th station ($i=1,\ldots,n$).

$
\begin{bmatrix}
1 & 0 & X_{1}^{(1)} & X_{2}^{(1)} & 0 & 0 \\
0 & 1 & 0 & 0 & X_{1}^{(1)} & X_{2}^{(1)} \\
1 & 0 & X_{1}^{(2)} & X_{2}^{(2)} & 0 & 0 \\
0 & 1 & 0 & 0 & X_{1}^{(2)} & X_{2}^{(2)} \\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
1 & 0 & X_{1}^{(n)} & X_{2}^{(n)} & 0 & 0 \\
0 & 1 & 0 & 0 & X_{1}^{(n)} & X_{2}^{(n)}
\end{bmatrix}
\begin{bmatrix}
t_{1} \\
t_{2} \\
\varepsilon_{11} \\
\varepsilon_{12} \\
\varepsilon_{21} \\
\varepsilon_{22}
\end{bmatrix}
=
\begin{bmatrix}
v_{1}^{(1)} \\
v_{2}^{(1)} \\
v_{1}^{(2)} \\
v_{2}^{(2)} \\
\vdots \\
v_{1}^{(n)} \\
v_{2}^{(n)} \\
\end{bmatrix}
$

Putting the above relationship in the form of $\mathbf{M}\mathbf{x} = \mathbf{y}$, we invert it for $\mathbf{x}$ as follows:

$\mathbf{x} = \left( \mathbf{M}^{T}\mathbf{W}\mathbf{M} \right)^{-1} \mathbf{M}^{T}\mathbf{W} \mathbf{y}$, 
where $\mathbf{W}$ is a diagonal matrix of weighting factors.
Among many possible choices for $\mathbf{W}$, we choose the Gaussian function of distance:

$ W = \exp \left( -\frac{(d^{(i)})^{2}}{2\alpha^{2}} \right)$, where $d^{(i)}$ is the distance from a location to the $i$-th GPS station and $\alpha$ is a distance weighting constant of 40 km.


### Generate a grid of points

In [ ]:
ngridpts = 51
lons = np.linspace(125.0, 131.0, ngridpts)
lats = np.linspace(33.0, 38.5, ngridpts)
long, latg = np.meshgrid(lons, lats)
positions = np.vstack([long.ravel(), latg.ravel()]).transpose()
positions.shape

### Load station locations

In [ ]:
try: 
    with open( 'Korea_GPS_Stations_locations.p', 'rb' ) as handle:
        Korea_GPS_Locations = pickle.load(handle)
        # This is a dictionary with 55 stations as keys
        # Each item is a lenth-2 numpy array.
        print(Korea_GPS_Locations['DAEJ'])
except FileNotFoundError:
    print("Failed to load the pickled data. Generate ELTM coefficients using GPSdata.ipynb.")

### Define functions for the main tasks

In [ ]:
def get_M_W( position, Korea_GPS_CoeffCov, Korea_GPS_Locations ):
    # Since v vector follows the order of stations stored in CoeffCov,
    # we loop over the station keys in CoeffCov to keep the order consistent. 
    nstations = len(Korea_GPS_CoeffCov.keys())
    M = np.zeros((2*nstations, 6))
    W = np.eye(2*nstations)
    counter = 0
    alpha_sq = 4.0e4**2 # (40 km)^2
     
#     from pyproj import Geod
#     geod = Geod(ellps="WGS84")
    from pyproj import Proj
    if position[0] >= 120.0 and position[0] < 126.0:
        utm_zone = 51
    elif position[0] >= 126.0 and position[0] < 132.0:
        utm_zone = 52
    p = Proj(proj='utm',zone=utm_zone, ellps='WGS84', preserve_units=False)
    # Convert position lon lat to UTM coordinates
    pos_x, pos_y = p(position[0], position[1])
    
    for station in Korea_GPS_CoeffCov.keys():
#         # Attempt 1: Compute geodesic distance from 'position' to station
#         lons = [pos_lon, station_lon]
#         lats = [pos_lat, station_lat]
#         d = geod.line_length(lons, lats)
#         # calcalate W value for this station
#         Wval = np.exp(-0.5*d**2/alpha_sq)
#         # compute along-latitude distance from 'position' to station
#         lons = [pos_lon, station_lon]
#         lats = [pos_lat, pos_lat]
#         X1 = geod.line_length(lons, lats)
#         # compute along-longitude distance from 'position' to station
#         lons = [pos_lon, pos_lon]
#         lats = [pos_lat, station_lat]
#         X2 = geod.line_length(lons, lats)

        # Convert station lon lat to UTM Coordinates
        station_x, station_y = p( Korea_GPS_Locations[station][0], Korea_GPS_Locations[station][1])
        # Compute distances using UTM coordinates
        X1 = station_x - pos_x
        X2 = station_y - pos_y
        d2 = X1**2 + X2**2
        #print(X1, X2, np.sqrt(d2))
        
        # Calcalate W value for this station
        Wval = np.exp(-0.5*d2/alpha_sq)
        
        # Populate M and W.
        M[2*counter,:] = [1.0, 0.0, X1, X2, 0.0, 0.0]
        M[2*counter+1,:] = [0.0, 1.0, 0.0, 0.0, X1, X2]
        W[2*counter,2*counter] = Wval
        W[2*counter+1,2*counter+1] = Wval
        counter = counter + 1
        
    return M, W

def get_eps_omg( M, W, vel_vector ):
    # Solve Ax = b
    tmpA = np.matmul(np.transpose(M), W)
    A = np.matmul( tmpA, M )
    b = np.dot(tmpA, vel_vector)
    # Take only the velocity gradient and reshape it to 2x2 matrix.
    vgrad = np.linalg.solve(A,b)[2:].reshape((2,2))
    eps = 0.5*(vgrad+np.transpose(vgrad))
    omg = 0.5*(vgrad-np.transpose(vgrad))
    return eps, omg

### Generate velocity vector from the ELTM coefficients

In [ ]:
def get_velocities( Korea_GPS_CoeffCov, d ):
    velocities = {}
    k = 2.0/365.0 # 1 over 0.5 year.
    for station in Korea_GPS_CoeffCov.keys():
        p = Korea_GPS_CoeffCov[station]['ecoeff'][0]
        a = Korea_GPS_CoeffCov[station]['ecoeff'][10]
        ve = p + k*a/(1.0+k*(d+21.0))
        p = Korea_GPS_CoeffCov[station]['ncoeff'][0]
        a = Korea_GPS_CoeffCov[station]['ncoeff'][10]
        vn = p + k*a/(1.0+k*(d+21.0))
        velocities[station] = np.array([ve,vn])
    return velocities

def get_vel_vector( vel ):
    vel_vector = np.zeros( 2*len(vel.keys()) )
    counter = 0
    for s in vel.keys():
        vel_vector[2*counter] = vel[s][0]
        vel_vector[2*counter+1] = vel[s][1]
        counter = counter + 1
    return vel_vector

### Get strain rates and rotation at every grid point

In [ ]:
# get vel velctor for April 12, 2011, one day after 04/11.
vel = get_velocities( Korea_GPS_CoeffCov, 1.0)
print(vel['DAEJ'])

vel_vector = get_vel_vector( vel )
for position in positions[0:2,:]:
    M, W = get_M_W( position, Korea_GPS_CoeffCov, Korea_GPS_Locations )  
    eps, omg = get_eps_omg( M, W, vel_vector )
    # print(eps, omg)